In [1]:
import pandas as pd
import altair as alt
import numpy as np

c:\Users\merii\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [24]:
df

,Fecha,Temperatura,Flujo,Max_Last_Six,First_Over_Max_Last_Six,baseline,higher_baseline,days_higher_than_baseline,ovulation_confirmed
0,2024-02-20,NaN,S,NaN,False,36.3,False,0.0,False
1,2024-02-21,NaN,S,NaN,False,36.3,False,0.0,False
2,2024-02-22,NaN,S,NaN,False,36.3,False,0.0,False
3,2024-02-23,NaN,S,NaN,False,36.3,False,0.0,False
4,2024-02-24,NaN,S,NaN,False,36.3,False,0.0,False
5,2024-02-25,NaN,S,NaN,False,36.3,False,0.0,False
6,2024-02-26,NaN,S,NaN,False,36.3,False,0.0,False
7,2024-02-27,36.20,f,NaN,False,36.3,False,0.0,False
8,2024-02-28,36.00,f,NaN,False,36.3,False,0.0,False
9,2024-02-29,36.00,F,NaN,False,36.3,False,0.0,False


In [20]:
df = pd.read_excel('../data/Marina_ciclo3.xlsx', engine = 'openpyxl' )

# Calculate the start_date and end_date to include at least 28 days
min_days = min(28, len(df))  # Ensure at least 28 days
if min_days<28:
    num_days = 28-min_days
    start_date = df['Fecha'].min()
    end_date = df['Fecha'].max() + pd.DateOffset(days=num_days)
else:
    start_date = df['Fecha'].min()
    end_date = df['Fecha'].max()
domain = [start_date, end_date]

In [25]:
end_date

Timestamp('2024-03-18 00:00:00')

In [21]:
df['Max_Last_Six'] = df['Temperatura'].rolling(window=6, min_periods=6).apply(lambda x: x[:-1].max())

mask = (df['Temperatura'] >= df['Max_Last_Six']) & ~(df['Temperatura'].shift(1) >= df['Max_Last_Six'].shift(1))

# Find the index of the first occurrence
first_occurrence_index = mask.idxmax()

# Set all values to False except for the first occurrence
df['First_Over_Max_Last_Six'] = False
df.loc[first_occurrence_index, 'First_Over_Max_Last_Six'] = True

# Get that temperature
baseline = df[df.First_Over_Max_Last_Six == True].Max_Last_Six.iloc[0]
if np.isnan(baseline):
    baseline=40

df['baseline'] = baseline


# Check the next three temperatures are higher than that value
df['higher_baseline']=df.apply(lambda x: ((x.Temperatura >= x.baseline+0.1)), axis=1)

df['days_higher_than_baseline'] = df['higher_baseline'].rolling(window=3, min_periods = 1).sum()

# confirm ovulation
df['ovulation_confirmed'] = False
mask_ovulation = ((df.ovulation_confirmed == False) & (df.days_higher_than_baseline==3))
first_occurrence_ovulation = mask_ovulation.idxmax()
df.loc[first_occurrence_ovulation::, 'ovulation_confirmed'] = True

if df.loc[0, 'ovulation_confirmed']:
    df.loc[0, 'ovulation_confirmed'] = False

In [22]:
df['Fecha'] = pd.to_datetime(df['Fecha'])

# Plot
line = alt.Chart(df).mark_line().encode(
    x='Fecha:T',
    y=alt.Y('Temperatura:Q', scale=alt.Scale(domain=[36, 37]))
)

points = alt.Chart(df).mark_point().encode(
    x='Fecha:T',
    y='Temperatura:Q',
    tooltip=['Fecha:T', 'Temperatura:Q']
)

baseline = alt.Chart(df).mark_line().encode(
    x='Fecha:T',
    y=alt.Y('baseline:Q', scale=alt.Scale(domain=[36, 37]))
)

points_ovulation = alt.Chart(df[df['ovulation_confirmed']]).mark_point(color='green', filled = True, size = 100).encode(
    x='Fecha:T',
    y='Temperatura:Q',
    tooltip=['Fecha:T', 'ovulation_confirmed']
)

chart = (line + points+baseline+points_ovulation).properties(
    title='Temperatura vs Fecha',
    width=600,
    height=300
).encode(
    x=alt.X('Fecha:T', scale=alt.Scale(domain=domain))
)

chart.configure_axis(
    labelAngle=45
)

alt.LayerChart(...)